## PEMS03 EDA

In [6]:
import pandas as pd 
import numpy as np
import sys
import os
import torch
sys.path.append(os.path.abspath('..'))

In [7]:
df = pd.read_csv('../data/PEMS03/data.csv', parse_dates=['timestamp'])
df.head()

,timestamp,313344,313349,313438,313450,313487,313493,313512,313551,313552,...,313144,313159,313166,313172,313178,313184,313190,313197,313204,313339
0,2018-09-01 00:00:00,20.0,20.0,182.0,182.0,91.0,182.0,136.0,91.0,91.0,...,158.0,63.0,63.0,63.0,125.0,114.0,63.0,63.0,115.0,63.0
1,2018-09-01 00:05:00,22.0,22.0,174.0,174.0,87.0,174.0,131.0,87.0,87.0,...,158.0,62.0,62.0,62.0,144.0,109.0,62.0,63.0,109.0,62.0
2,2018-09-01 00:10:00,22.0,22.0,183.0,183.0,92.0,183.0,139.0,92.0,92.0,...,160.0,57.0,57.0,57.0,131.0,116.0,57.0,57.0,115.0,57.0
3,2018-09-01 00:15:00,50.0,49.0,137.0,139.0,60.0,158.0,111.0,57.0,55.0,...,174.0,109.0,30.0,58.0,106.0,102.0,52.0,111.0,88.0,48.0
4,2018-09-01 00:20:00,37.0,35.0,128.0,123.0,54.0,131.0,111.0,46.0,52.0,...,162.0,77.0,31.0,55.0,109.0,127.0,49.0,109.0,76.0,47.0


In [19]:
print(df.shape)
print(df.columns)
print(type(df['timestamp']))
# 26208 registros , 358 estacões

(26208, 359)
Index(['timestamp', '313344', '313349', '313438', '313450', '313487', '313493',
       '313512', '313551', '313552',
       ...
       '313144', '313159', '313166', '313172', '313178', '313184', '313190',
       '313197', '313204', '313339'],
      dtype='object', length=359)
<class 'pandas.core.series.Series'>


In [28]:
# Print columns with missing values if any, else print 'No missing values'
missing = df.isna().sum()
if (missing > 0).any():
    print(missing[missing > 0])
else:
    print('No missing values')

No missing values


## Temporal embedding testing

In [8]:
from embeddings.temporal import TemporalEmbedding,SinusoidalTemporalEncoding

# Create the embedding
print(f'original - {type(df['timestamp'].iloc[0])}')
print(f'original - {[df['timestamp'].iloc[2016]]}')


# Instantiate temporal embedding
temb = SinusoidalTemporalEncoding(df['timestamp'])

all_embs = torch.stack(list(temb.embeddings.values())).cpu().numpy()
print(f'All embeddings shape: {all_embs.shape}')


original - <class 'pandas._libs.tslibs.timestamps.Timestamp'>
original - [Timestamp('2018-09-08 00:00:00')]
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])
torch.Size([2, 2])

In [9]:
print(f'All embeddings: {all_embs[0]}')

All embeddings: [[ 0.67596895 -0.9984653 ]
 [ 0.73693013 -0.05538072]]


In [4]:
import torch
import numpy as np

# Stack all embeddings into a single tensor
all_embs = torch.stack(list(temb.embeddings.values())).cpu().numpy()

# Find duplicates
_, idx, counts = np.unique(all_embs, axis=0, return_index=True, return_counts=True)
has_duplicates = np.any(counts > 1)
if has_duplicates:
    print("There are duplicate embeddings.")
    dup_vals = all_embs[np.where(counts > 1)[0]]
    for val in dup_vals:
        lines = np.where((all_embs == val).all(axis=1))[0]
        print(f"Duplicate embedding: {val}, at lines: {lines}")
else:
    print("All embeddings are unique.")

All embeddings are unique.


In [ ]:
d_model=4
base_time = pd.Timestamp("2010-01-01 00:00:00")
dt = pd.Timestamp("2018-09-26 05:45:00")
seconds = (dt - base_time).total_seconds()
pos = torch.tensor([[seconds]])
i = torch.arange(d_model // 2).unsqueeze(0)
div_term = torch.pow(10000, (2 * i) / d_model)

angle_rads = pos / div_term
print(f"Angle radians: {angle_rads}")
# Compute sin and cos parts
# Note: torch.sin and torch.cos expect radians, so we don't need to convert
sin_part = torch.sin(angle_rads)
print(f"Sin part: {sin_part}")
# Compute cos part  
cos_part = torch.cos(angle_rads)

print(f"Cos part: {cos_part}")
# Stack sin and cos parts

# Squeeze to remove the extra dimension
# Note: torch.stack expects a list of tensors, so we wrap sin_part and cos_part
# in a list and then stack them along a new dimension.
# The result will have shape (2, d_model // 2) and we squeeze it
# to get a shape of (d_model // 2,).
# This is the final output of the sinusoidal encoding.
# The output will be a tensor of shape (d_model // 2,).
# If d_model is 4, the output will be a tensor of shape (2, 2).
# If d_model is 6, the output will be a tensor of shape (3, 2).
# If d_model is 8, the output will be a tensor of shape (4, 2).
# If d_model is 10, the output will be a tensor of shape (5, 2).
# If d_model is 12, the output will be a tensor of shape (6, 2).
# If d_model is 14, the output will be a tensor of shape (7, 2).
# If d_model is 16, the output will be a tensor of shape (8, 2).
# If d_model is 18, the output will be a tensor of shape (9, 2).
# If d_model is 20, the output will be a tensor of shape (10, 2).
# torch.stack([sin_part, cos_part]).squeeze()
print(torch.cat([sin_part, cos_part], dim=-1).view(-1).size())

Angle radians: tensor([[2.7564e+08, 2.7564e+06]])
Sin part: tensor([[-0.3730,  0.4085]])
Cos part: tensor([[ 0.9278, -0.9128]])
torch.Size([4])


In [3]:

import torch
tokeni = torch.tensor([-0.5725, -0.0053,  0.0101,  0.0050, -0.7881,  0.1757, -0.8966,  0.9844,
         0.4428, -0.5935,  0.0118,  0.0101,  0.0050, -0.7881,  0.1757, -0.8966,
         0.9844,  0.4428, -0.5935,  0.0118,  0.0101,  0.0050, -1.2056,  0.1757,
        -0.8966,  0.9844,  0.4428, -0.5544, -0.0202,  0.0101,  0.0050, -0.6768,
         0.1757, -0.8966,  0.9844,  0.4428, -0.5544, -0.0202,  0.0101,  0.0050,
        -1.1778,  0.1757, -0.8966,  0.9844,  0.4428])

tokeni = tokeni.reshape(1, 5, 9)

print(tokeni)

tensor([[[-0.5725, -0.0053,  0.0101,  0.0050, -0.7881,  0.1757, -0.8966,
           0.9844,  0.4428],
         [-0.5935,  0.0118,  0.0101,  0.0050, -0.7881,  0.1757, -0.8966,
           0.9844,  0.4428],
         [-0.5935,  0.0118,  0.0101,  0.0050, -1.2056,  0.1757, -0.8966,
           0.9844,  0.4428],
         [-0.5544, -0.0202,  0.0101,  0.0050, -0.6768,  0.1757, -0.8966,
           0.9844,  0.4428],
         [-0.5544, -0.0202,  0.0101,  0.0050, -1.1778,  0.1757, -0.8966,
           0.9844,  0.4428]]])
